In [71]:
import requests
import json
import datetime
import pandas as pd

USER_ID = "vitorhugoma"
TOKEN = "~~"


def check_empty_values(df):
    # Check if dataframe is empty
    if df.empty:
        print("No songs downloaded. Finishing execution")
    else:
        return df
    #return False
def check_primary_key_unique(df):
    # Primary Key Check
    if pd.Series(df['played_at']).is_unique:
        return df
    else:
        raise Exception("Primary Key check is violated")

def check_for_nulls(df):
    # Check for nulls
    if df.isnull().values.any():
        raise Exception("Null values found")
    else:
        return df

def check_yesterday_played_songs(df):
    today = datetime.datetime.now()
    yesterday = today - datetime.timedelta(days=1)
    for day in range(len(df)):
        if df['timestamp'][day] != yesterday.strftime("%Y-%m-%d"):
            df = df.drop(day)
    return df

def check_if_valid_data(df):
    check_empty_values(df)
    check_primary_key_unique(df)
    check_for_nulls(df)
    return True

    
if __name__ == "__main__":

# Extract part of the ETL process
    
#     headers = spotify.get_resource_header()
    headers = {
       "Accept" : "application/json",
       "Content-Type" : "application/json",
       "Authorization" : "Bearer {token}".format(token=TOKEN)
    }

    # Convert time to Unix timestamp in miliseconds
    today = datetime.datetime.now()
    yesterday = today - datetime.timedelta(days=1)
    yesterday_unix_timestamp = int(yesterday.timestamp())

    # Download all songs you've listened to "after yesterday", which means in the last 24 hours
    r = requests.get("https://api.spotify.com/v1/me/player/recently-played?limit=50&after={time}".format(time=yesterday_unix_timestamp), headers = headers)

    data = r.json()
    song_names = []
    artist_names = []
    played_at_list = []
    timestamps = []
    print(data)
    # Extracting only the relevant bits of data from the json object
    for song in data["items"]:
        song_names.append(song["track"]["name"])
        artist_names.append(song["track"]["album"]["artists"][0]["name"])
        played_at_list.append(song["played_at"])
        timestamps.append(song["played_at"][0:10])

    # Prepare a dictionary in order to turn it into a pandas dataframe below
    song_dict = {
        "song_name" : song_names,
        "artist_name": artist_names,
        "played_at" : played_at_list,
        "timestamp" : timestamps
    }

    song_df = pd.DataFrame(song_dict, columns = ["song_name", "artist_name", "played_at", "timestamp"])
    # Checking the music played yesterday
    song_df = check_yesterday_played_songs(song_df)
    #Validating the data
    if check_if_valid_data(song_df):
        print("Data valid, proceed to Load stage")

{'items': [{'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6Ae0wz09vBFYZXJ2bJAKUl'}, 'href': 'https://api.spotify.com/v1/artists/6Ae0wz09vBFYZXJ2bJAKUl', 'id': '6Ae0wz09vBFYZXJ2bJAKUl', 'name': 'Kiaz', 'type': 'artist', 'uri': 'spotify:artist:6Ae0wz09vBFYZXJ2bJAKUl'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'LY', 'MA', 'MC', 'MD', 'ME', 'MG', 

In [72]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker
import sqlite3


DATABASE_LOCATION = "sqlite:///my_played_tracks123.sqlite"

# Load

engine = sqlalchemy.create_engine(DATABASE_LOCATION)
conn = sqlite3.connect('my_played_tracks.sqlite')
cursor = conn.cursor()

sql_query = """
CREATE TABLE IF NOT EXISTS my_played_tracks(
song_name VARCHAR(200),
artist_name VARCHAR(200),
played_at VARCHAR(200),
timestamp VARCHAR(200),
CONSTRAINT primary_key_constraint PRIMARY KEY (played_at)
)
"""

cursor.execute(sql_query)
print("Opened database successfully")

try:
    song_df.to_sql("my_played_tracks", engine, index=False, if_exists='append')
except:
    print("Data already exists in the database")

conn.close()
print("Close database successfully")

Opened database successfully
Close database successfully
